In [66]:
import numpy as np
import pandas as pd
import re
import pickle
import matplotlib.pyplot as plt

class TF_IDF:
    def __init__(self, documents):
        self.documents = documents
        self.raw_tokens = [re.findall("[A-Za-z0-9\-]+", document, re.IGNORECASE) for document in documents]
        self.tokens = [[raw_token.lower() for raw_token in raw_tokens] for raw_tokens in self.raw_tokens]

        TFs = self.TF()
        IDF = self.IDF()
        TF_ID = {}
        for TF in TFs:
            for k, v in TF.items():
                if k in IDF:
                    TF_ID[k] = IDF[k] * v
        self.vocab = TF_ID

    
    def TF(self): #
        total_counts = []
        counts = {}
        for tokens in self.tokens:
            for token in tokens:
                if token in counts:
                    counts[token] += 1
                else:
                    counts[token] = 1
            total_counts.append(counts)
            counts = {}
        Total_TFs = []
        TFs = {}
        for counts in total_counts:
            TFs = {k: v / sum(counts.values()) for k, v in counts.items()}
            Total_TFs.append(TFs)
            TFs = {}
        return Total_TFs
        
    def IDF(self): #document 
        counts = {}
        unique_tokens = []
        for token in self.tokens:
            unique_tokens.append(list(set(token)))
        for i in range(len(unique_tokens)):
            for token in unique_tokens[i]:
                if token in counts:
                    counts[token] += 1
                else:
                    counts[token] = 1
            return counts
        IDFs = {k: np.log(len(self.documents) / v) for k, n in counts.items()}
        return IDFs
        
    def TFIDF(self):
        TF_IDFs = []
        TFs = self.TF()
        IDF = self.IDF()
        for TF in TFs:
            TF_ID = {}
            for k, v in TF.items():
                if k in IDF:
                    TF_ID[k] = IDF[k] * v
            TF_IDFs.append(TF_ID)

        TF_IDF_df = pd.DataFrame(TF_IDFs).fillna(0)
        TF_IDF_matrix = TF_IDF_df.to_numpy()
        return TF_IDF_matrix

    def transform(self, document):
        raw_tokens = re.findall("[A-Za-z0-9\-]+", document, re.IGNORECASE)
        tokens = [raw_token.lower() for raw_token in raw_tokens]

        token = [self.vocab[token] if token in self.vocab else 0 for token in tokens]
        token_vector = [[self.vocab][key] if value in token else 0 for key, value in self.vocab.items()]
        token_vector = np.array(token_vector)
        return token_vector


documents = pd.read_csv("training_data_2.csv")
size = len(documents["message"])
label = documents["label"][:int(size * 0.8)]
train = documents["message"][:int(size * 0.8)]
test = documents["message"][int(size * 0.2):]
label_t = documents["label"][int(size * 0.2):]
vectorizer_t = TF_IDF(test)
X_test = vectorizer_t.TFIDF()
print(X_test)
vectorizer = TF_IDF(train)
X = vectorizer.TFIDF()

f = lambda x: 1 if x == "spam" else 0
y = list(map(f, label))
y = np.array(y)

class LogisticRegression:
    def __init__(self, X, W, y):
        self.y = y
        self.W = W
        self.X = X
        self.loss_history = []

    def sigmoid(self, z): #check in internet
        return 1/(1 + np.exp(-z))

    def nll(self): #check in internet, I've copyed the formula
        self.y_pred = self.sigmoid(self.X @ self.W)
        return -np.sum(self.y * np.log(self.y_pred) + (1 - self.y) * np.log(1 - self.y_pred))
        
    def predict(self, X):
        y_pred = self.sigmoid(X[:, :self.W.shape[0]] @ self.W)
        preds = ["spam" if pred > 0.5 else "ham" for pred in y_pred]
        return preds

    def fit(self, epochs, lr): #gradient descent
        for i in range(epochs):
            self.y_pred = self.sigmoid(self.X @ self.W)
            gradient = -np.dot(self.X.T, (self.y - self.y_pred)) / self.y.size
            self.W -= lr * gradient
            self.loss_history.append(self.nll())

            if i % 100 == 0:
                print(f"Epoch: {i}, NLL: {self.nll()}")
    def plot_loss_curve(self):
        plt.plot(self.loss_history)
        plt.xlabel("Epoch")
        plt.ylabel("NLL")
        plt.title("Loss Curve")
        plt.show()
        
#try:
    print("Loading model")
    with open("email_classifier_2.pkl", "rb") as f:
        model = pickle.load(f)
        #X_t = vectorizer.transform("Warranted")
        print(model.predict(X_test) == label_t)
            # X_t = vectorizer_t
# except:
#      print("Training model")
#      W = np.random.randn(X.shape[1])
#      #print(X.shape, W.shape) just for getting if our parameters are compatible
#      model = LogisticRegression(X, W, y)
#      model.fit(1000, 0.1)
#      model.plot_loss_curve()
    
#      with open("email_classifier_2.pkl", "wb") as f:
#          pickle.dump(model, f)

[[0.1        0.1        0.1        0.1        0.1        0.1
  0.1        0.1        0.1        0.1       ]
 [0.         0.         0.         0.         0.11111111 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.1        0.        ]
 [0.         0.         0.         0.         0.16666667 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.14285714 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.08333333 0.08333333 0.
  0.         0.         0.08333333 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.125      0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.125      0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0